<a href="https://colab.research.google.com/github/hazelchiang2102/ADALL_github/blob/main/Lab/ML_Project_Draft_1601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Core libraries
import pandas as pd
import numpy as np
# Visualisation
import matplotlib.pyplot as plt
# Modelling and preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [6]:
import pandas as pd
import io

csv_data = """
Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0
0.0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0
0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0
1.0,1.0,1.0,1.0,30.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0
0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0
1.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,13.0,6.0,8.0
0.0,1.0,1.0,1.0,34.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,30.0,1.0,0.0,10.0,5.0,1.0
0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,15.0,0.0,0.0,7.0,5.0,7.0
1.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,4.0,6.0
0.0,0.0,1.0,1.0,33.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,4.0,30.0,28.0,0.0,0.0,4.0,6.0,2.0
0.0,1.0,0.0,1.0,33.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,5.0,0.0,0.0,0.0,6.0,6.0,8.0
0.0,1.0,1.0,1.0,21.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,4.0,3.0
1.0,0.0,0.0,1.0,23.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,7.0,5.0,6.0
0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,15.0,0.0,0.0,0.0,2.0,6.0,7.0
0.0,0.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,10.0,0.0,0.0,1.0,4.0,6.0,8.0
0.0,1.0,1.0,1.0,22.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,30.0,0.0,1.0,0.0,12.0,4.0,4.0
0.0,1.0,1.0,1.0,38.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,15.0,30.0,1.0,0.0,13.0,2.0,3.0
0.0,0.0,0.0,1.0,28.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,7.0,0.0,1.0,5.0,5.0,5.0
1.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,5.0,4.0
0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,6.0,0.0,1.0,0.0,9.0,4.0,6.0
0.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,6.0,8.0
1.0,1.0,1.0,1.0,37.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,1.0,1.0,10.0,6.0,5.0
1.0,1.0,1.0,1.0,28.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,12.0,2.0,4.0
1.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,20.0,20.0,1.0,0.0,8.0,4.0,7.0
0.0,0.0,1.0,1.0,31.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
1.0,1.0,1.0,1.0,34.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,7.0,1.0,0.0,9.0,5.0,4.0
0.0,1.0,0.0,1.0,33.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,13.0,3.0,3.0
0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,6.0,4.0,8.0
0.0,1.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,6.0,2.0
1.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,12.0,3.0,3.0
0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,9.0,4.0,4.0
0.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,3.0,1.0,1.0,8.0,4.0,3.0
0.0,0.0,0.0,1.0,22.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,10.0,0.0,0.0,12.0,5.0,7.0
0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,4.0,5.0,3.0
0.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,0.0,1.0,9.0,3.0,1.0
0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,4.0,6.0,7.0
1.0,1.0,0.0,1.0,31.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,5.0,0.0,0.0,13.0,4.0,4.0
0.0,0.0,0.0,1.0,28.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,4.0,15.0,30.0,1.0,0.0,7.0,4.0,3.0
0.0,1.0,0.0,1.0,23.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,25.0,0.0,0.0,0.0,9.0,6.0,8.0
0.0,0.0,1.0,1.0,31.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0
0.0,0.0,1.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,30.0,0.0,0.0,0.0,8.0,6.0,3.0
0.0,0.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,7.0,5.0,2.0
0.0,1.0,1.0,1.0,25.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,10.0,4.0,7.0
0.0,1.0,1.0,1.0,37.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,30.0,2.0,1.0,1.0,10.0,5.0,6.0
1.0,1.0,1.0,1.0,33.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,30.0,1.0,0.0,11.0,4.0,2.0
0.0,1.0,1.0,1.0,30.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,10.0,17.0,1.0,0.0,9.0,4.0,1.0
0.0,1.0,0.0,1.0,22.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,3.0,3.0
1.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,0.0,10.0,4.0,5.0
0.0,0.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,10.0,5.0,6.0
0.0,0.0,1.0,1.0,34.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,4.0,6.0,8.0
0.0,0.0,0.0,1.0,29.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,0.0,1.0,11.0,6.0,8.0
0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,0.0,0.0,6.0,6.0,8.0
0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,6.0,8.0
0.0,1.0,1.0,1.0,33.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,1.0,11.0,6.0,6.0
0.0,0.0,0.0,1.0,20.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,4.0,5.0
1.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,11.0,4.0,4.0
0.0,1.0,0.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,5.0,1.0,0.0,11.0,4.0,2.0
0.0,1.0,0.0,1.0,34.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,9.0,6.0,5.0
0.0,1.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,5.0,1.0,0.0,0.0,7.0,5.0,8.0
0.0,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,30.0,30.0,1.0,0.0,10.0,4.0,3.0
0.0,1.0,1.0,1.0,38.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,10.0,5.0,1.0,1.0,11.0,5.0,6.0
0.0,0.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,8.0,5.0,7.0
0.0,1.0,0.0,1.0,25.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,7.0,1.0,0.0,13.0,4.0,4.0
1.0,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,10.0,3.0,2.0
1.0,1.0,1.0,1.0,23.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,7.0,5.0,3.0
0.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,12.0,5.0,5.0
1.0,1.0,1.0,1.0,27.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,4.0,2.0,0.0,1.0,7.0,6.0,8.0
0.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,4.0,3.0
0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,5.0,7.0
0.0,0.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,5.0,4.0
0.0,0.0,0.0,1.0,31.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,4.0,0.0,1.0,6.0,5.0,6.0
1.0,1.0,0.0,1.0,35.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,5.0,8.0
0.0,1.0,1.0,1.0,28.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,10.0,1.0,0.0,12.0,6.0,2.0
0.0,1.0,1.0,1.0,26.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,4.0,0.0,0.0,1.0,11.0,6.0,3.0
0.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,8.0
0.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,7.0,6.0,8.0
0.0,1.0,1.0,1.0,33.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,3.0,0.0,0.0,1.0,7.0,4.0,7.0
0.0,1.0,1.0,1.0,23.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,1.0,13.0,4.0,4.0
1.0,0.0,1.0,1.0,32.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,13.0,2.0,2.0
0.0,0.0,0.0,1.0,27.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,4.0,7.0
1.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,2.0,30.0,1.0,0.0,9.0,6.0,8.0
0.0,0.0,1.0,1.0,22.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,7.0,2.0,0.0,0.0,1.0,5.0,8.0
0.0,1.0,0.0,1.0,39.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,1.0,8.0,4.0,8.0
0.0,0.0,0.0,1.0,30.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,10.0,0.0,0.0,1.0,5.0,6.0,8.0
1.0,1.0,1.0,1.0,31.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,13.0,5.0,6.0
0.0,1.0,1.0,1.0,32.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,5.0,15.0,0.0,0.0,10.0,5.0,8.0
0.0,0.0,0.0,1.0,22.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,3.0,6.0,8.0
1.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,5.0,7.0
1.0,0.0,1.0,1.0,29.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,30.0,10.0,1.0,0.0,11.0,5.0,2.0
0.0,0.0,0.0,1.0,31.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,6.0,7.0
1.0,1.0,1.0,1.0,25.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,15.0,30.0,1.0,0.0,9.0,2.0,3.0
1.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,4.0,0.0,3.0,0.0,0.0,3.0,5.0,3.0
0.0,1.0,0.0,1.0,45.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,4.0,6.0
0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,11.0,4.0,3.0
0.0,1.0,1.0,1.0,35.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,7.0,4.0,8.0
0.0,1.0,0.0,1.0,27.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,6.0,8.0
0.0,0.0,1.0,1.0,19.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,5.0
0.0,1.0,1.0,1.0,23.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,6.0,6.0
1.0,1.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,9.0,6.0,7.0
0.0,0.0,1.0,1.0,29.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,6.0,2.0
0.0,1.0,0.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,30.0,0.0,0.0,10.0,6.0,6.0
0.0,0.0,1.0,1.0,22.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,30.0,0.0,0.0,0.0,8.0,5.0,2.0
1.0,1.0,1.0,1.0,32.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,4.0,5.0
0.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,10.0,0.0,0.0,10.0,6.0,7.0
0.0,1.0,1.0,1.0,33.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,15.0,1.0,0.0,7.0,5.0,3.0
0.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,30.0,19.0,0.0,0.0,4.0,5.0,6.0
0.0,1.0,1.0,1.0,26.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,20.0,1.0,0.0,13.0,6.0,4.0
0.0,0.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,4.0,6.0
0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,8.0,6.0,8.0
0.0,0.0,1.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,8.0,6.0,6.0
0.0,0.0,0.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,3.0,30.0,30.0,1.0,1.0,3.0,5.0,6.0
0.0,0.0,0.0,1.0,24.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,6.0,7.0
0.0,0.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,6.0,4.0,7.0
0.0,1.0,1.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,20.0,4.0,0.0,0.0,7.0,5.0,7.0
0.0,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,6.0,6.0
0.0,1.0,1.0,1.0,40.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,8.0
1.0,0.0,0.0,1.0,29.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,10.0,0.0,0.0,7.0,6.0,7.0
0.0,0.0,1.0,1.0,30.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,15.0,7.0,1.0,0.0,10.0,4.0,4.0
0.0,1.0,1.0,1.0,31.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,7.0,0.0,1.0,10.0,4.0,8.0
0.0,0.0,1.0,1.0,39.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,1.0,5.0,5.0,7.0
0.0,0.0,0.0,0.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,10.0,0.0,0.0,0.0,6.0,3.0,7.0
0.0,1.0,1.0,1.0,25.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,30.0,1.0,1.0,12.0,5.0,5.0
0.0,1.0,1.0,1.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,3.0,3.0,0.0,1.0,7.0,4.0,4.0
0.0,0.0,1.0,1.0,35.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,7.0,4.0,8.0
0.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,1.0,13.0,4.0,3.0
0.0,0.0,0.0,1.0,31.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,4.0,8.0
0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,15.0,0.0,0.0,0.0,8.0,5.0,6.0
0.0,1.0,1.0,1.0,32.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0,0.0,20.0,1.0,0.0,11.0,4.0,2.0
0.0,0.0,1.0,1.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,30.0,30.0,0.0,0.0,8.0,3.0,4.0
0.0,1.0,1.0,1.0,23.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,5.0,0.0,0.0,12.0,6.0,8.0
0.0,0.0,1.0,1.0,22.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,10.0,15.0,0.0,0.0,11.0,4.0,5.0
0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,6.0,8.0
1.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,4.0,6.0
0.0,0.0,1.0,1.0,27.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,5.0,6.0,8.0
0.0,1.0,1.0,1.0,31.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,6.0,1.0,0.0,9.0,5.0,7.0
0.0,0.0,0.0,1.0,23.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,10.0,6.0,0.0,1.0,1.0,3.0,3.0
0.0,1.0,1.0,1.0,28.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,8.0,10.0,1.0,0.0,9.0,4.0,2.0
0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,11.0,4.0,8.0
1.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,6.0,5.0
1.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,8.0,5.0,8.0
0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,12.0,3.0,1.0
0.0,1.0,1.0,1.0,39.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,15.0,3.0,1.0,1.0,9.0,3.0,3.0
0.0,0.0,0.0,1.0,34.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,5.0,6.0
0.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,5.0,0.0,0.0,0.0,11.0,4.0,4.0
0.0,0.0,0.0,1.0,22.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,12.0,4.0,6.0
1.0,1.0,1.0,1.0,29.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,3.0,0.0,0.0,8.0,4.0,7.0
0.0,1.0,1.0,1.0,33.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,12.0,4.0,6.0
1.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,12.0,5.0,4.0
1.0,1.0,0.0,1.0,33.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,5.0,3.0,30.0,1.0,0.0,8.0,5.0,6.0
0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,21.0,0.0,1.0,0.0,10.0,5.0,6.0
0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,6.0,8.0
1.0,1.0,1.0,1.0,47.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,6.0,5.0
0.0,0.0,1.0,1.0,21.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,13.0,4.0,8.0
1.0,1.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,14.0,14.0,0.0,0.0,9.0,6.0,5.0
0.0,1.0,0.0,1.0,18.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,0.0,8.0,6.0,4.0
1.0,0.0,1.0,1.0,34.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,1.0,10.0,3.0,5.0
0.0,0.0,0.0,1.0,22.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,9.0,4.0,8.0
0.0,0.0,0.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,4.0,1.0,1.0,1.0,8.0,6.0,8.0
1.0,1.0,0.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2.0,2.0,0.0,0.0,8.0,4.0,8.0
0.0,0.0,0.0,0.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,4.0,4.0
1.0,1.0,0.0,1.0,34.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,5.0,1.0,0.0,10.0,5.0,6.0
0.0,0.0,1.0,1.0,29.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,15.0,1.0,0.0,8.0,5.0,4.0
0.0,1.0,1.0,1.0,37.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,10.0,6.0,7.0
0.0,0.0,1.0,1.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,6.0,7.0
0.0,1.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,12.0,5.0,6.0
1.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,10.0,5.0,6.0
1.0,1.0,1.0,1.0,28.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,30.0,1.0,0.0,12.0,5.0,3.0
0.0,0.0,1.0,1.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,26.0,4.0,0.0,0.0,7.0,6.0,8.0
0.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,0.0,7.0,5.0,5.0
0.0,1.0,1.0,1.0,29.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,0.0,10.0,5.0,2.0
0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,1.0,4.0,5.0,5.0
1.0,1.0,0.0,1.0,32.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,15.0,1.0,1.0,10.0,5.0,8.0
1.0,1.0,1.0,1.0,37.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,13.0,6.0,7.0
0.0,1.0,1.0,1.0,29.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,4.0,0.0,1.0,8.0,5.0,6.0
0.0,0.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,6.0,0.0,1.0,7.0,5.0,7.0
0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,3.0,5.0,0.0,0.0,7.0,6.0,8.0
0.0,1.0,1.0,1.0,34.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,0.0,13.0,4.0,3.0
0.0,0.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,6.0,8.0
0.0,1.0,1.0,1.0,26.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,11.0,4.0,5.0
0.0,1.0,1.0,1.0,37.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,30.0,0.0,0.0,1.0,9.0,4.0,6.0
0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,8.0,6.0,8.0
0.0,1.0,1.0,1.0,38.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,6.0,8.0
0.0,0.0,1.0,1.0,29.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,5.0,1.0,1.0,8.0,4.0,6.0
0.0,1.0,0.0,1.0,43.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,7.0,1.0,0.0,10.0,5.0,2.0
0.0,1.0,1.0,1.0,23.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,9.0,6.0,8.0
0.0,1.0,0.0,1.0,35.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,1.0,1.0,9.0,2.0,5.0
0.0,0.0,0.0,1.0,24.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,1.0,30.0,0.0,1.0,9.0,6.0,7.0
0.0,1.0,0.0,1.0,23.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,10.0,5.0,5.0
0.0,1.0,1.0,1.0,34.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,4.0,8.0
1.0,1.0,0.0,1.0,33.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,11.0,6.0,5.0
0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,9.0,6.0,7.0
0.0,0.0,0.0,1.0,29.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,12.0,4.0,7.0
1.0,1.0,1.0,1.0,29.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,15.0,15.0,0.0,0.0,12.0,4.0,4.0
0.0,0.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,4.0,1.0,0.0,0.0,8.0,5.0,8.0
0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,6.0,8.0
0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,5.0,2.0,0.0,0.0,6.0,6.0,8.0
1.0,1.0,1.0,1.0,55.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,15.0,0.0,0.0,11.0,5.0,2.0
0.0,0.0,0.0,1.0,22.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,7.0,6.0,8.0
0.0,1.0,1.0,1.0,49.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,15.0,1.0,0.0,0.0,9.0,6.0,6.0
0.0,1.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,8.0
0.0,0.0,0.0,1.0,27.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0,5.0,4.0
0.0,0.0,0.0,1.0,27.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,1.0,13.0,6.0,6.0
0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,1.0,6.0,6.0,8.0
0.0,0.0,1.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,13.0,4.0,3.0
0.0,1.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,5.0,8.0
1.0,1.0,1.0,1.0,31.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,4.0,2.0,0.0,0.0,9.0,5.0,6.0
0.0,0.0,0.0,1.0,34.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,3.0,0.0,0.0,1.0,0.0,6.0,6.0,7.0
0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,3.0,7.0,2.0,0.0,1.0,10.0,5.0,6.0
0.0,1.0,0.0,1.0,18.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,5.0,30.0,7.0,0.0,0.0,4.0,4.0,1.0
0.0,0.0,0.0,1.0,18.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,3.0,4.0
0.0,0.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,6.0,6.0,7.0
0.0,1.0,0.0,1.0,42.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,1.0,8.0,4.0,7.0
1.0,1.0,1.0,1.0,28.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,12.0,4.0,1.0
0.0,1.0,1.0,1.0,31.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,4.0,7.0,12.0,1.0,1.0,9.0,4.0,1.0
1.0,1.0,0.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,5.0,4.0
0.0,0.0,0.0,1.0,22.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,5.0
0.0,1.0,1.0,1.0,17.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,30.0,0.0,1.0,0.0,8.0,4.0,7.0
0.0,1.0,0.0,1.0,31.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,30.0,1.0,0.0,12.0,6.0,6.0
0.0,1.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,8.0,5.0,8.0
0.0,0.0,0.0,1.0,22.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,15.0,1.0,0.0,0.0,6.0,6.0,7.0
0.0,0.0,0.0,1.0,40.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,14.0,0.0,0.0,5.0,5.0,8.0
0.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,12.0,5.0,4.0
0.0,0.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,10.0,6.0,8.0
0.0,1.0,0.0,1.0,33.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,5.0,0.0,0.0,1.0,10.0,6.0,8.0
0.0,1.0,0.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0,5.0,0.0,0.0,0.0,12.0,6.0,4.0
0.0,0.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,4.0,10.0,30.0,1.0,0.0,7.0,5.0,2.0
0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,5.0,8.0
0.0,0.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,11.0,6.0,7.0
0.0,0.0,1.0,1.0,26.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,20.0,5.0,1.0,0.0,8.0,5.0,6.0
0.0,0.0,1.0,1.0,32.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,9.0,6.0,8.0
1.0,1.0,0.0,1.0,38.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,15.0,30.0,1.0,1.0,8.0,5.0,6.0
0.0,1.0,1.0,1.0,33.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,30.0,30.0,1.0,0.0,12.0,6.0,6.0
1.0,1.0,1.0,1.0,24.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,12.0,6.0,7.0
1.0,1.0,1.0,1.0,32.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,28.0,1.0,1.0,8.0,4.0,8.0
0.0,1.0,1.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,11.0,6.0,8.0
0.0,0.0,1.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,10.0,2.0,0.0,1.0,11.0,6.0,6.0
0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,10.0,5.0,8.0
0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,13.0,4.0,6.0
1.0,0.0,0.0,1.0,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,6.0,5.0,7.0
0.0,1.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,4.0,8.0
0.0,1.0,0.0,1.0,36.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,5.0,6.0
0.0,0.0,0.0,1.0,32.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,6.0,5.0,8.0
0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,7.0,5.0,7.0
0.0,0.0,0.0,1.0,23.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,4.0,2.0
1.0,1.0,1.0,1.0,26.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,10.0,6.0,7.0
1.0,1.0,1.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,13.0,5.0,6.0
0.0,1.0,1.0,1.0,21.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,5.0,5.0,3.0,1.0,0.0,13.0,3.0,2.0
0.0,0.0,1.0,1.0,23.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,9.0,6.0,7.0
0.0,0.0,0.0,1.0,23.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,12.0,3.0,3.0
"""
df_inline_data = pd.read_csv(io.StringIO(csv_data))
print("Successfully loaded inline data into df_inline_data:")
print(df_inline_data.head())


Successfully loaded inline data into df_inline_data:
   Diabetes_binary  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0              0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1              0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2              0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3              0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4              0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0

In [7]:
df_inline_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       254 non-null    float64
 1   HighBP                254 non-null    float64
 2   HighChol              254 non-null    float64
 3   CholCheck             254 non-null    float64
 4   BMI                   254 non-null    float64
 5   Smoker                254 non-null    float64
 6   Stroke                254 non-null    float64
 7   HeartDiseaseorAttack  254 non-null    float64
 8   PhysActivity          254 non-null    float64
 9   Fruits                254 non-null    float64
 10  Veggies               254 non-null    float64
 11  HvyAlcoholConsump     254 non-null    float64
 12  AnyHealthcare         254 non-null    float64
 13  NoDocbcCost           254 non-null    float64
 14  GenHlth               254 non-null    float64
 15  MentHlth              2

In [ ]:
# Example: Replace this with the raw URL of your GitHub file
github_raw_url = 'https://raw.githubusercontent.com/rq-goh/ADALL_github/main/laptop_prices_2024_sgd_TL.csv'
try:
    df = pd.read_csv(github_raw_url)
    print("Successfully loaded data from GitHub!")
    display(df.head())
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please ensure the URL is correct and the file format is compatible with `pd.read_csv`.")

In [ ]:
#read in the dataset (select 2015)
year = '2015'
brfss_2015_dataset = pd.read_csv('/content/sample_data/2015.csv')

In [ ]:
#How many rows and columns
brfss_2015_dataset.shape

(171142, 330)

In [ ]:
#check that the data loaded in is in the correct format
pd.set_option('display.max_columns', 500)
brfss_2015_dataset.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,PVTRESD1,COLGHOUS,STATERES,CELLFON3,LADULT,NUMADULT,NUMMEN,NUMWOMEN,CTELNUM1,CELLFON2,CADULT,PVTRESD2,CCLGHOUS,CSTATE,LANDLINE,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,BLOODCHO,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,DIABAGE2,SEX,MARITAL,EDUCA,RENTHOM1,NUMHHOL2,NUMPHON2,CPDEMO1,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,PREGNANT,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,SMOKDAY2,STOPSMK2,LASTSMK2,USENOW3,ALCDAY5,AVEDRNK2,DRNK3GE5,MAXDRNKS,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXRACT11,EXEROFT1,EXERHMM1,EXRACT21,EXEROFT2,EXERHMM2,STRENGTH,LMTJOIN3,ARTHDIS2,ARTHSOCL,JOINPAIN,SEATBELT,FLUSHOT6,FLSHTMY2,IMFVPLAC,PNEUVAC3,HIVTST6,HIVTSTD3,WHRTST10,PDIABTST,PREDIAB1,INSULIN,BLDSUGAR,FEETCHK2,DOCTDIAB,CHKHEMO3,FEETCHK,EYEEXAM,DIABEYE,DIABEDU,PAINACT2,QLMENTL2,QLSTRES2,QLHLTH2,CAREGIV1,CRGVREL1,CRGVLNG1,CRGVHRS1,CRGVPRB1,CRGVPERS,CRGVHOUS,CRGVMST2,CRGVEXPT,VIDFCLT2,VIREDIF3,VIPRFVS2,VINOCRE2,VIEYEXM2,VIINSUR2,VICTRCT4,VIGLUMA2,VIMACDG2,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,WTCHSALT,LONGWTCH,DRADVISE,ASTHMAGE,ASATTACK,ASERVIST,ASDRVIST,ASRCHKUP,ASACTLIM,ASYMPTOM,ASNOSLEP,ASTHMED3,ASINHALR,HAREHAB1,STREHAB1,CVDASPRN,ASPUNSAF,RLIVPAIN,RDUCHART,RDUCSTRK,ARTTODAY,ARTHWGT,ARTHEXER,ARTHEDU,TETANUS,HPVADVC2,HPVADSHT,SHINGLE2,HADMAM,HOWLONG,HADPAP2,LASTPAP2,HPVTEST,HPLSTTST,HADHYST2,PROFEXAM,LENGEXAM,BLDSTOOL,LSTBLDS3,HADSIGM3,HADSGCO1,LASTSIG3,PCPSAAD2,PCPSADI1,PCPSARE1,PSATEST1,PSATIME,PCPSARS1,PCPSADE1,PCDMDECN,SCNTMNY1,SCNTMEL1,SCNTPAID,SCNTWRK1,SCNTLPAD,SCNTLWK1,SXORIENT,TRNSGNDR,RCSGENDR,RCSRLTN2,CASTHDX2,CASTHNO2,EMTSUPRT,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY,ADEAT1,ADFAIL,ADTHINK,ADMOVE,MISTMNT,ADANXEV,QSTVER,QSTLANG,EXACTOT1,EXACTOT2,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_CHISPNC,_CRACE1,_CPRACE,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PA1MIN_,PAVIG11_,PAVIG21_,PA1VIGM_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,1.0,NaN,1.0,2.0,NaN,3.0,1.000000e+00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,15.0,18.0,10.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,3.0,NaN,2.0,1.0,4.0,1.0,2.0,NaN,1.0,2.0,8.0,88.0,3.0,2.0,280.0,510.0,NaN,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,NaN,2.0,3.0,888.0,NaN,NaN,NaN,305.0,310.0,320.0,310.0,305.0,101.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,1.0,1.0,6.0,1.0,1.0,112014.0,1.0,1.0,1.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0,b'',b'',3.0,11011.0,28.78156,3.0,86.344681,NaN,NaN,NaN,NaN,1.0,0.614125,341.384853,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,9.0,1.0,63.0,5.0,70.0,178.0,12701.0,4018.0,4.0,2.0,1.0,2.0

In [ ]:
# select specific columns
brfss_df_selected = brfss_2015_dataset[['DIABETE3',
                                         '_RFHYPE5',
                                         'TOLDHI2', '_CHOLCHK',
                                         '_BMI5',
                                         'SMOKE100',
                                         'CVDSTRK3', '_MICHD',
                                         '_TOTINDA',
                                         '_FRTLT1', '_VEGLT1',
                                         '_RFDRHV5',
                                         'HLTHPLN1', 'MEDCOST',
                                         'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK',
                                         'SEX', '_AGEG5YR', 'EDUCA', 'INCOME2' ]]

In [ ]:
brfss_df_selected.shape

(171142, 22)

In [ ]:
brfss_df_selected.head()

,DIABETE3,_RFHYPE5,TOLDHI2,_CHOLCHK,_BMI5,SMOKE100,CVDSTRK3,_MICHD,_TOTINDA,_FRTLT1,_VEGLT1,_RFDRHV5,HLTHPLN1,MEDCOST,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,SEX,_AGEG5YR,EDUCA,INCOME2
0,3.0,2.0,1.0,1.0,4018.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,5.0,18.0,15.0,1.0,2.0,9.0,4.0,3.0
1,3.0,1.0,2.0,2.0,2509.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0,88.0,88.0,2.0,2.0,7.0,6.0,1.0
2,3.0,1.0,1.0,1.0,2204.0,NaN,1.0,NaN,9.0,9.0,9.0,9.0,1.0,2.0,4.0,88.0,15.0,NaN,2.0,11.0,4.0,99.0
3,3.0,2.0,1.0,1.0,2819.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,5.0,30.0,30.0,1.0,2.0,9.0,4.0,8.0
4,3.0,1.0,2.0,1.0,2437.0,2.0,2.0,2.0,2.0,9.0,1.0,1.0,1.0,2.0,5.0,88.0,20.0,2.0,2.0,9.0,5.0,77.0


In [ ]:
#Drop Missing Values - knocks 100,000 rows out right away
brfss_df_selected = brfss_df_selected.dropna()
brfss_df_selected.shape

(133150, 22)

In [ ]:
# DIABETE3
# going to make this ordinal. 0 is for no diabetes or only during pregnancy, 1 is for pre-diabetes or borderline diabetes, 2 is for yes diabetes
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_df_selected['DIABETE3'] = brfss_df_selected['DIABETE3'].replace({2:0, 3:0, 1:2, 4:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE3 != 9]
brfss_df_selected.DIABETE3.unique()

array([0., 2., 1.])

In [ ]:
#1 _RFHYPE5
#Change 1 to 0 so it represetnts No high blood pressure and 2 to 1 so it represents high blood pressure
brfss_df_selected['_RFHYPE5'] = brfss_df_selected['_RFHYPE5'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFHYPE5 != 9]
brfss_df_selected._RFHYPE5.unique()

array([1., 0.])

In [ ]:
#2 TOLDHI2
# Change 2 to 0 because it is No
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_df_selected['TOLDHI2'] = brfss_df_selected['TOLDHI2'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI2 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI2 != 9]
brfss_df_selected.TOLDHI2.unique()

array([1., 0.])

In [ ]:
#3 _CHOLCHK
# Change 3 to 0 and 2 to 0 for Not checked cholesterol in past 5 years
# Remove 9
brfss_df_selected['_CHOLCHK'] = brfss_df_selected['_CHOLCHK'].replace({3:0,2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._CHOLCHK != 9]
brfss_df_selected._CHOLCHK.unique()

array([1., 0.])

In [ ]:
#4 _BMI5 (no changes, just note that these are BMI * 100. So for example a BMI of 4018 is really 40.18)
brfss_df_selected['_BMI5'] = brfss_df_selected['_BMI5'].div(100).round(0)
brfss_df_selected._BMI5.unique()

array([40., 25., 28., 24., 27., 30., 26., 23., 34., 33., 21., 22., 31.,
       38., 20., 19., 32., 46., 41., 37., 36., 29., 35., 18., 54., 45.,
       39., 47., 43., 55., 49., 42., 17., 16., 48., 44., 50., 59., 15.,
       52., 53., 57., 51., 14., 58., 63., 61., 56., 60., 74., 62., 64.,
       13., 66., 73., 65., 68., 85., 71., 84., 67., 70., 82., 79., 92.,
       72., 88., 96., 81., 12., 77., 95., 75., 91., 69., 76., 87., 89.,
       83., 98., 86.])

In [ ]:
#5 SMOKE100
# Change 2 to 0 because it is No
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_df_selected['SMOKE100'] = brfss_df_selected['SMOKE100'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 9]
brfss_df_selected.SMOKE100.unique()

array([1., 0.])

In [ ]:
#7 _MICHD
#Change 2 to 0 because this means did not have MI or CHD
brfss_df_selected['_MICHD'] = brfss_df_selected['_MICHD'].replace({2: 0})
brfss_df_selected._MICHD.unique()

array([0., 1.])

In [ ]:
#8 _TOTINDA
# 1 for physical activity
# change 2 to 0 for no physical activity
# Remove all 9 (don't know/refused)
brfss_df_selected['_TOTINDA'] = brfss_df_selected['_TOTINDA'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._TOTINDA != 9]
brfss_df_selected._TOTINDA.unique()

array([0., 1.])

In [ ]:
#9 _FRTLT1
# Change 2 to 0. this means no fruit consumed per day. 1 will mean consumed 1 or more pieces of fruit per day
# remove all dont knows and missing 9
brfss_df_selected['_FRTLT1'] = brfss_df_selected['_FRTLT1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._FRTLT1 != 9]
brfss_df_selected._FRTLT1.unique()

array([0., 1.])

In [ ]:
#10 _VEGLT1
# Change 2 to 0. this means no vegetables consumed per day. 1 will mean consumed 1 or more pieces of vegetable per day
# remove all dont knows and missing 9
brfss_df_selected['_VEGLT1'] = brfss_df_selected['_VEGLT1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._VEGLT1 != 9]
brfss_df_selected._VEGLT1.unique()

array([1., 0.])

In [ ]:
#11 _RFDRHV5
# Change 1 to 0 (1 was no for heavy drinking). change all 2 to 1 (2 was yes for heavy drinking)
# remove all dont knows and missing 9
brfss_df_selected['_RFDRHV5'] = brfss_df_selected['_RFDRHV5'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFDRHV5 != 9]
brfss_df_selected._RFDRHV5.unique()

array([0., 1.])

In [ ]:
#12 HLTHPLN1
# 1 is yes, change 2 to 0 because it is No health care access
# remove 7 and 9 for don't know or refused
brfss_df_selected['HLTHPLN1'] = brfss_df_selected['HLTHPLN1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.HLTHPLN1 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.HLTHPLN1 != 9]
brfss_df_selected.HLTHPLN1.unique()

array([1., 0.])

In [ ]:
#13 MEDCOST
# Change 2 to 0 for no, 1 is already yes
# remove 7 for don/t know and 9 for refused
brfss_df_selected['MEDCOST'] = brfss_df_selected['MEDCOST'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST != 9]
brfss_df_selected.MEDCOST.unique()

array([0., 1.])

In [ ]:
#14 GENHLTH
# This is an ordinal variable that I want to keep (1 is Excellent -> 5 is Poor)
# Remove 7 and 9 for don't know and refused
brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 9]
brfss_df_selected.GENHLTH.unique()

array([5., 3., 2., 4., 1.])

In [ ]:
#15 MENTHLTH
# already in days so keep that, scale will be 0-30
# change 88 to 0 because it means none (no bad mental health days)
# remove 77 and 99 for don't know not sure and refused
brfss_df_selected['MENTHLTH'] = brfss_df_selected['MENTHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 99]
brfss_df_selected.MENTHLTH.unique()

array([18.,  0., 30.,  3.,  5., 15., 10.,  6., 20.,  2., 25.,  1., 29.,
        4.,  7.,  8., 21., 14., 26.,  9., 16., 28., 11., 12., 23., 24.,
       17., 13., 27., 19., 22.])

In [ ]:
#16 PHYSHLTH
# already in days so keep that, scale will be 0-30
# change 88 to 0 because it means none (no bad mental health days)
# remove 77 and 99 for don't know not sure and refused
brfss_df_selected['PHYSHLTH'] = brfss_df_selected['PHYSHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 99]
brfss_df_selected.PHYSHLTH.unique()

array([15.,  0., 30.,  2., 14., 28.,  7., 20.,  3., 10.,  1.,  5., 17.,
        4., 19.,  6., 21., 12.,  8., 25., 27., 22., 29., 24.,  9., 16.,
       18., 23., 13., 26., 11.])

In [ ]:
#17 DIFFWALK
# change 2 to 0 for no. 1 is already yes
# remove 7 and 9 for don't know not sure and refused
brfss_df_selected['DIFFWALK'] = brfss_df_selected['DIFFWALK'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 9]
brfss_df_selected.DIFFWALK.unique()

array([1., 0.])

In [ ]:
#18 SEX
# in other words - is respondent male (somewhat arbitrarily chose this change because men are at higher risk for heart disease)
# change 2 to 0 (female as 0). Male is 1
brfss_df_selected['SEX'] = brfss_df_selected['SEX'].replace({2:0})
brfss_df_selected.SEX.unique()

array([0., 1.])

In [ ]:
#19 _AGEG5YR
# already ordinal. 1 is 18-24 all the way up to 13 wis 80 and older. 5 year increments.
# remove 14 because it is don't know or missing
brfss_df_selected = brfss_df_selected[brfss_df_selected._AGEG5YR != 14]
brfss_df_selected._AGEG5YR.unique()

array([ 9.,  7., 11., 10., 13.,  8.,  4.,  6.,  2., 12.,  5.,  1.,  3.])

In [ ]:
#20 EDUCA
# This is already an ordinal variable with 1 being never attended school or kindergarten only up to 6 being college 4 years or more
# Scale here is 1-6
# Remove 9 for refused:
brfss_df_selected = brfss_df_selected[brfss_df_selected.EDUCA != 9]
brfss_df_selected.EDUCA.unique()

array([4., 6., 3., 5., 2., 1.])

In [ ]:
#21 INCOME2
# Variable is already ordinal with 1 being less than $10,000 all the way up to 8 being $75,000 or more
# Remove 77 and 99 for don't know and refused
brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME2 != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME2 != 99]
brfss_df_selected.INCOME2.unique()

array([3., 1., 8., 6., 4., 7., 2., 5.])

In [ ]:
#Check the shape of the dataset now: We have 253,680 cleaned rows and 22 columns (1 of which is our dependent variable)
brfss_df_selected.shape

(98210, 22)

In [ ]:
#Let's see what the data looks like after Modifying Values
brfss_df_selected.head()

,DIABETE3,_RFHYPE5,TOLDHI2,_CHOLCHK,_BMI5,SMOKE100,CVDSTRK3,_MICHD,_TOTINDA,_FRTLT1,_VEGLT1,_RFDRHV5,HLTHPLN1,MEDCOST,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,SEX,_AGEG5YR,EDUCA,INCOME2
0,0.0,1.0,1.0,1.0,40.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,28.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,27.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,24.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
#Check Class Sizes of the heart disease column
brfss_df_selected.groupby(['DIABETE3']).size()

,0
DIABETE3,
0.0,82571
1.0,1861
2.0,13778


In [ ]:
#Rename the columns to make them more readable#Rename the columns to make them more readable
brfss = brfss_df_selected.rename(columns = {'DIABETE3':'Diabetes_012',
                                         '_RFHYPE5':'HighBP',
                                         'TOLDHI2':'HighChol', '_CHOLCHK':'CholCheck',
                                         '_BMI5':'BMI',
                                         'SMOKE100':'Smoker',
                                         'CVDSTRK3':'Stroke', '_MICHD':'HeartDiseaseorAttack',
                                         '_TOTINDA':'PhysActivity',
                                         '_FRTLT1':'Fruits', '_VEGLT1':"Veggies",
                                         '_RFDRHV5':'HvyAlcoholConsump',
                                         'HLTHPLN1':'AnyHealthcare', 'MEDCOST':'NoDocbcCost',
                                         'GENHLTH':'GenHlth', 'MENTHLTH':'MentHlth', 'PHYSHLTH':'PhysHlth', 'DIFFWALK':'DiffWalk',
                                         'SEX':'Sex', '_AGEG5YR':'Age', 'EDUCA':'Education', 'INCOME2':'Income' })

In [ ]:
brfss.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,28.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,27.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,24.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
brfss.shape

(98210, 22)

In [ ]:
#Check how many respondents have no diabetes, prediabetes or diabetes. Note the class imbalance!
brfss.groupby(['Diabetes_012']).size()

,0
Diabetes_012,
0.0,82571
1.0,1861
2.0,13778


In [ ]:
#************************************************************************************************
brfss.to_csv('diabetes_012_health_indicators_BRFSS2015.csv', sep=",", index=False)
#************************************************************************************************